In [ ]:
from qiskit import *
import networkx as nx
import numpy as np
from qiskit.optimization.ising import max_cut
#from qiskit.aqua.input import EnergyInput
#from qiskit.aqua.algorithms import VQE
#from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms.adaptive import QAOA
from qiskit.aqua.components.optimizers import L_BFGS_B

In [ ]:
# Definimos y dibujamos el grafo

n=5 
G=nx.Graph()
G.add_nodes_from(np.arange(0,n,1))
elist=[(0,1,1.0),(0,2,1.0),(1,2,1.0),(1,3,1.0),(2,4,1.0),(3,4,1.0)]
G.add_weighted_edges_from(elist)
pos = nx.spring_layout(G)
nx.draw_networkx(G,  node_size=600, pos = pos )

In [ ]:
# Calculamos la matriz de adyacencia y el hamiltoniano

w = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight'] 
print(w)

qubitOp, offset = max_cut.get_operator(w)


In [ ]:
optimizer = L_BFGS_B()
p = 2
qaoa = QAOA(qubitOp, optimizer, p)

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)
result = qaoa.run(quantum_instance)


x = max_cut.sample_most_likely(result['eigvecs'][0])
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

colors = ['r' if max_cut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, pos=pos)